In [6]:
import os
import shutil
import random
import yaml

# パスの設定
base_dir = r'C:\Users\CorneAI\YOLOv11_modified_Mobius'
dataset_dir = os.path.join(base_dir, 'dataset')
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# datasetフォルダが存在する場合は削除して新規作成
if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)
os.makedirs(dataset_dir)

# train, validフォルダの作成
train_img_dir = os.path.join(dataset_dir, 'train', 'images')
train_label_dir = os.path.join(dataset_dir, 'train', 'labels')
valid_img_dir = os.path.join(dataset_dir, 'valid', 'images')
valid_label_dir = os.path.join(dataset_dir, 'valid', 'labels')

os.makedirs(train_img_dir)
os.makedirs(train_label_dir)
os.makedirs(valid_img_dir)
os.makedirs(valid_label_dir)

# 画像ファイル名のリストを取得
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)  # ランダムに並び替え

# 8:2の比率で分割するインデックスを計算
split_idx = int(len(image_files) * 0.8)
train_files = image_files[:split_idx]
valid_files = image_files[split_idx:]

# ファイルをtrainとvalidに分配
for file in train_files:
    # 画像ファイルのコピー
    src_img = os.path.join(images_dir, file)
    dst_img = os.path.join(train_img_dir, file)
    shutil.copy2(src_img, dst_img)
    
    # 対応するラベルファイルの処理
    # 拡張子を除いたファイル名を取得
    base_name = os.path.splitext(file)[0]
    label_file = base_name + '.txt'
    src_label = os.path.join(labels_dir, label_file)
    
    # ラベルファイルが存在する場合のみコピー
    if os.path.exists(src_label):
        dst_label = os.path.join(train_label_dir, label_file)
        shutil.copy2(src_label, dst_label)
    else:
        print(f"警告: ラベルファイルが見つかりません: {label_file}")

for file in valid_files:
    # 画像ファイルのコピー
    src_img = os.path.join(images_dir, file)
    dst_img = os.path.join(valid_img_dir, file)
    shutil.copy2(src_img, dst_img)
    
    # 対応するラベルファイルの処理
    base_name = os.path.splitext(file)[0]
    label_file = base_name + '.txt'
    src_label = os.path.join(labels_dir, label_file)
    
    # ラベルファイルが存在する場合のみコピー
    if os.path.exists(src_label):
        dst_label = os.path.join(valid_label_dir, label_file)
        shutil.copy2(src_label, dst_label)
    else:
        print(f"警告: ラベルファイルが見つかりません: {label_file}")

# yamlファイルの作成
yaml_path = os.path.join(dataset_dir, 'data.yaml')
yaml_content = {
    'train': os.path.join(dataset_dir, 'train', 'images'),
    'val': os.path.join(dataset_dir, 'valid', 'images'),
    'nc': 1,  # クラス数
    'names': {0: 'pupil',}  # クラス名
}

with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

print("データセットの準備が完了しました。")
print(f"訓練データ: {len(train_files)}ファイル")
print(f"検証データ: {len(valid_files)}ファイル")
print(f"YAMLファイル作成: {yaml_path}")

データセットの準備が完了しました。
訓練データ: 2833ファイル
検証データ: 709ファイル
YAMLファイル作成: C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\data.yaml


In [13]:
# GPUが使用可能かどうかを確認
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    device_name = torch.cuda.get_device_name(0) if device_count > 0 else "なし"
    print(f"GPUは使用可能です。検出されたGPUデバイス数: {device_count}")
    print(f"GPUデバイス名: {device_name}")
    print(f"現在のデバイス: {torch.cuda.current_device()}")
    print(f"CUDA バージョン: {torch.version.cuda}")
else:
    print("GPUは使用できません。CPUが使用されます。")

# メモリ情報の表示（GPUが利用可能な場合）
if torch.cuda.is_available():
    print("\nGPUメモリ情報:")
    print(f"割り当てられたメモリ: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"キャッシュされたメモリ: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
    print(f"最大割り当てメモリ: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")


GPUは使用できません。CPUが使用されます。


In [ ]:
from ultralytics import YOLO
import wget

# モデルのロード（初回時に自動的にダウンロード）
wget.download('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-obb.pt')
model = YOLO('yolo11n-obb.pt')

In [12]:
results = model.train(data=r"C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\data.yaml", epochs=1000, imgsz=640)


Ultralytics 8.3.99  Python-3.10.5 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i9-12900H)
engine\trainer: task=obb, mode=train, model=yolo11n-obb.pt, data=C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\data.yaml, epochs=1000, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\train\labels... 2833 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2833/2833 [00:05<00:00, 516.92it/s]


train: New cache created: C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\train\labels.cache


val: Scanning C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\valid\labels... 709 images, 0 backgrounds, 0 corrupt: 100%|██████████| 709/709 [00:01<00:00, 510.37it/s]


val: New cache created: C:\Users\CorneAI\YOLOv11_modified_Mobius\dataset\valid\labels.cache
Plotting labels to runs\obb\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 87 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\obb\train
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/1000         0G      2.277      6.469      2.516         31        640:   3%|▎         | 5/178 [00:16<09:29,  3.29s/it]


KeyboardInterrupt: 